Plots augmentations for MLCN article

In [47]:
from soma import aims
import numpy as np
import torch

from contrastive.augmentations import RemoveRandomBranchTensor, PartialCutOutTensor_Roll

# Plots data without augmentation

In [29]:
vol = aims.read("/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Rlabels/103010_cropped_foldlabel.nii.gz")
skel = aims.read("/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Rcrops/103010_cropped_skeleton.nii.gz")

In [30]:
np.unique(vol.np, return_counts=True)

(array([   0,  212, 1001, 1005, 1061, 1079, 1089, 1104, 1132, 1154, 1166,
        1207, 1214, 1226, 1249, 1257, 1298, 1310, 1323, 1327, 1345, 2001,
        2005, 2061, 2062, 2079, 2089, 2104, 2154, 2166, 2181, 2207, 2214,
        2226, 2249, 2257, 2298, 2310, 2323, 2342, 2345, 3259, 3357, 3365,
        3372, 3739, 3754, 3864, 4108, 4117, 4370, 4375, 4376, 4382, 4383,
        4390, 4407, 4600, 4686, 4916, 5349, 5361], dtype=int16),
 array([24890,     1,    21,   107,    15,    54,     4,    39,     2,
           36,    45,     3,     8,    18,    12,    10,    17,     3,
           23,     1,   116,     9,    30,     7,     1,    19,     4,
           18,    17,    13,     1,     8,    12,    16,     9,     6,
            6,     5,     7,     8,    24,     3,    17,     7,    17,
           28,     6,    10,     9,     4,    15,    31,     3,    14,
           20,     4,     1,     1,     1,     2,     1,     1]))

In [31]:
np.unique(skel.np, return_counts=True)

(array([  0,  30,  60, 110], dtype=int16), array([24986,   229,   618,     7]))

In [32]:
%rm /tmp/bck_*.bck

In [33]:
def colorize(k, max=400):
    if k//1000 == 1: # surface simple
        return [0, (k%1000 / max+0.5), 0, 1]
    elif k//1000 == 2: # bottom
        return [k%1000 / max+0.5, 0, 0, 1]
    # elif k//1000 == 3:
    #     return [k%1000 / max, 0, 0, 1]
    # elif k//1000 == 4: # bottom
    #     return [k%1000 / max, 0, 0, 1]
    else:
        return [0, k%1000 / max + 0.5, 0, 1]

In [34]:
def colorize_skel(k):
    if k == 30: # surface simple
        return [1, 0, 0, 1]
    elif k == 60:
        return [0, 1, 0, 1]
    else:
        return [0, 0, 1, 1]

In [35]:
def bucketize_foldlabel(vol, root='bck'):
    c = aims.RawConverter_rc_ptr_Volume_S16_BucketMap_VOID(True)
    bck = aims.BucketMap()
    c.convert(vol, bck)
    for k in bck.keys():
        b2 = aims.BucketMap()
        b2.header().update(bck.header())
        b2[0] = bck[k]
        b2.header()['material'] = {'diffuse': colorize(k)}
        aims.write(b2, f"/tmp/{root}_{k}.bck")   

In [36]:

def bucketize_skeleton(skel, root='bck_skel'):
    c = aims.RawConverter_rc_ptr_Volume_S16_BucketMap_VOID(True)
    bck_skel = aims.BucketMap()
    c.convert(skel, bck_skel)
    
    for k in bck_skel.keys():
        b2 = aims.BucketMap()
        b2.header().update(bck_skel.header())
        b2[0] = bck_skel[k]
        b2.header()['material'] = {'diffuse': colorize_skel(k)}
        aims.write(b2, f"/tmp/{root}_{k}.bck")
    

In [37]:
bucketize_foldlabel(vol)


In [38]:
bucketize_skeleton(skel)

# Plots data with foldlabel augmentation

In [39]:
vol.shape

(17, 40, 38, 1)

In [40]:
branch_clipping = RemoveRandomBranchTensor(torch.from_numpy(vol.np), 40, (17,40,38,1), True, False)

In [41]:
branch_clipped = branch_clipping(torch.from_numpy(skel.np))

In [42]:
np.unique(branch_clipped.numpy(), return_counts=True)

(array([  0.,  30.,  60., 110.], dtype=float32),
 array([25333,   225,   275,     7]))

In [43]:
branch_clipped.shape

torch.Size([17, 40, 38, 1])

In [44]:
skel_clipped = aims.Volume(np.array(branch_clipped.numpy().astype(np.int16)))

In [45]:
bucketize_skeleton(skel_clipped, 'bck_skel_clipped')

In [46]:
skel_clipped

# Plots data with cut-out augmentation

In [181]:
cutting_out = PartialCutOutTensor_Roll(True, True, (1,12,25,25), False, None)

In [182]:
cutout = cutting_out(torch.from_numpy(skel.np))

In [183]:
np.unique(cutout.numpy(), return_counts=True)

(array([  0.,  30.,  60., 110.], dtype=float32),
 array([25302,   229,   305,     4]))

In [184]:
vol_cutout = aims.Volume(np.array(cutout.numpy().astype(np.int16)))

In [185]:
bucketize_skeleton(vol_cutout, 'bck_skel_cutout')

In [186]:
vol.shape

(17, 40, 38, 1)